In [3]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('../../data/pokemon.csv')

In [ ]:
image_folder = '../../data/images'

def get_image_path(pokemon_id):
    path = os.path.join(image_folder, f'{pokemon_id}.png')
    return path if os.path.exists(path) else None

In [5]:
df['image_path'] = df['Name'].apply(get_image_path)

In [8]:
# Check for missing images
missing = df[df['image_path'].isna()]
if not missing.empty:
    print(f"Warning: {len(missing)} images not found")
    print(missing['Name'].tolist())
    

In [9]:
print(df.head())

         Name  Type1   Type2   Evolution             image_path
0   bulbasaur  Grass  Poison     ivysaur   images\bulbasaur.png
1     ivysaur  Grass  Poison    venusaur     images\ivysaur.png
2    venusaur  Grass  Poison         NaN    images\venusaur.png
3  charmander   Fire     NaN  charmeleon  images\charmander.png
4  charmeleon   Fire     NaN   charizard  images\charmeleon.png


In [ ]:
# Import libraries for image processing
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Check a sample image - original size
sample_path = df['image_path'].iloc[0]
sample_img = Image.open(sample_path)
print(f"Original image mode: {sample_img.mode}")
print(f"Original image size: {sample_img.size}")
print(f"Original shape as array: {np.array(sample_img).shape}")
print(f"\nWill resize to: (224, 224) for EfficientNet-B0")
sample_img

In [ ]:
import sys
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), '..'))

from src.utils.preprocessing import get_transform, RGBAToRGB
from src.utils.dataset import PokemonDataset

# The manual load_image_as_rgb function has been replaced by a
# torchvision.transforms pipeline in src/utils/preprocessing.py:
#   RGBAToRGB()  -> composites RGBA onto white background
#   Resize(224)  -> LANCZOS resampling for EfficientNet-B0
#   ToTensor()   -> converts to float32 tensor in [0, 1]

In [ ]:
# Test the transforms pipeline on a sample image
transform = get_transform()
sample_img = Image.open(sample_path)
sample_tensor = transform(sample_img)

print(f"Tensor shape: {sample_tensor.shape}")  # Should be (3, 224, 224)
print(f"Dtype: {sample_tensor.dtype}")
print(f"Value range: [{sample_tensor.min():.2f}, {sample_tensor.max():.2f}]")

# Display the converted and resized image (permute CHW -> HWC for matplotlib)
plt.imshow(sample_tensor.permute(1, 2, 0))
plt.title(f"{df['Name'].iloc[0]} - Type: {df['Type1'].iloc[0]}\n(Resized to 224x224 for EfficientNet-B0)")
plt.axis('off')
plt.show()

In [ ]:
# Load full dataset using PokemonDataset
dataset = PokemonDataset('../../data')

print(f"Dataset size: {len(dataset)}")
print(f"Number of classes: {len(dataset.label_names)}")
print(f"Classes: {dataset.label_names}")

In [ ]:
# Verify all images are standardized tensors of shape (3, 224, 224)
sample_img, sample_label = dataset[0]
print(f"Sample tensor shape: {sample_img.shape}")
print(f"Sample label: {sample_label} ({dataset.label_names[sample_label]})")
print(f"Dtype: {sample_img.dtype}")
print(f"Value range: [{sample_img.min():.2f}, {sample_img.max():.2f}]")

In [ ]:
# Visualize a sample of preprocessed images from the dataset
fig, axes = plt.subplots(3, 4, figsize=(12, 9))
axes = axes.ravel()

for i in range(12):
    img, label = dataset[i]
    # Permute CHW -> HWC for matplotlib
    axes[i].imshow(img.permute(1, 2, 0))
    axes[i].set_title(f"{dataset.label_names[label]}", fontsize=9)
    axes[i].axis('off')

plt.tight_layout()
plt.suptitle('Sample Pokemon Images (Preprocessed via torchvision)', y=1.02, fontsize=14)
plt.show()

## Summary

**Image Conversion & Preprocessing Complete**
- Dataset: 809 Pokemon images with 18 primary type classes
- Converted RGBA to RGB (white background for transparency)
- Resized all images to 224x224 (LANCZOS) for EfficientNet-B0
- Normalized pixel values to [0, 1] float32 tensors via `ToTensor()`
- Output shape: `(3, 224, 224)` per image

**Implementation** (moved from notebook to modules):
- `src/utils/preprocessing.py` — `RGBAToRGB` transform + `get_transform()` pipeline
- `src/utils/dataset.py` — `PokemonDataset` (PyTorch Dataset with labels)

**Next Steps:**
1. Create train/val/test splits (70/10/20)
2. Set up data augmentation pipeline
3. Implement training loop with EfficientNet-B0